In [1]:
import numpy as np
import pyvista as pv
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from scipy.spatial import ConvexHull

# Load and visualize a .obj file
def load_and_display_obj(file_path):
    # Read the .obj file
    mesh = pv.read(file_path)
    
    # Display the 3D object
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, color='white', show_edges=True)
    plotter.show()
    
    return mesh

# Reduce dimensions or filter relevant points using PCA
def reduce_dimensions(mesh, n_components=3):
    points = mesh.points  # Get the 3D points of the mesh
    pca = PCA(n_components=n_components)
    reduced_points = pca.fit_transform(points)
    
    # Visualize reduced data
    cloud = pv.PolyData(reduced_points)
    cloud.plot(render_points_as_spheres=True, point_size=5)
    
    return reduced_points, pca

# Filter by retaining only convex hull points
def filter_convex_hull(points):
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]
    return hull_points

# Mixture of Gaussian (MoG) for primitive detection
def fit_mixture_of_gaussians(points, n_components=4):
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    gmm.fit(points)
    labels = gmm.predict(points)
    
    return labels, gmm

# Visualize the results of GMM clustering
def visualize_clusters(points, labels):
    cloud = pv.PolyData(points)
    cloud["Cluster"] = labels
    cloud.plot(render_points_as_spheres=True, point_size=5)

# Main function to integrate all steps
def main():
    # Path to the .obj file
    obj_file_path = "/home/ritz/Desktop/SEM 3/ML/Project/bottle.obj"  # Replace with your file path
    
    # Step 1: Load and display the .obj file
    mesh = load_and_display_obj(obj_file_path)
    
    # Step 2: Reduce dimensions or filter relevant points
    reduced_points, _ = reduce_dimensions(mesh)
    filtered_points = filter_convex_hull(reduced_points)
    
    # Step 3: Fit Gaussian Mixture Model
    labels, gmm = fit_mixture_of_gaussians(filtered_points, n_components=4)
    
    # Visualize the clusters
    visualize_clusters(filtered_points, labels)

if __name__ == "__main__":
    main()


Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x78173f6a2de0_0&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x78173b8cd5b0_1&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x781742d9b500_2&reconnect=auto" class="pyvi…

 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions
 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions
 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions
 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions
 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions


In [3]:
import numpy as np
import pyvista as pv
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from scipy.spatial import ConvexHull
from scipy.spatial.transform import Rotation as R
import open3d as o3d

def load_and_display_obj(file_path):
    """Load and display a .obj file."""
    mesh = pv.read(file_path)
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, color='white', show_edges=True)
    plotter.show()
    return mesh

def reduce_dimensions(mesh, n_components=3):
    """Reduce dimensions of the 3D points using PCA."""
    points = mesh.points  # Get the 3D points of the mesh
    pca = PCA(n_components=n_components)
    reduced_points = pca.fit_transform(points)
    
    # Visualize reduced data
    cloud = pv.PolyData(reduced_points)
    cloud.plot(render_points_as_spheres=True, point_size=5)
    return reduced_points, pca

def filter_convex_hull(points):
    """Filter points by retaining only those on the convex hull."""
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]
    return hull_points

def detect_primitives_o3d(points):
    """Detect shape primitives using Open3D."""
    primitives = []
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)

    # Plane detection using RANSAC
    plane_model, inliers = point_cloud.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)
    primitives.append(("Plane", plane_model))

    # Sphere detection (example, requires tuning)
    # Currently Open3D doesn't directly support sphere detection, but a similar approach can be implemented.
    # You can fit other primitives like cylinders or cones using a similar method.

    return primitives

def fit_mixture_of_gaussians(points, n_components=4):
    """Fit a Gaussian Mixture Model to the points."""
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    gmm.fit(points)
    labels = gmm.predict(points)
    return labels, gmm

def visualize_clusters(points, labels):
    """Visualize the clusters with colors."""
    cloud = pv.PolyData(points)
    cloud["Cluster"] = labels
    cloud.plot(render_points_as_spheres=True, point_size=5)

def generate_robotic_grasp_plan(primitives):
    """Generate a grasp plan for a robotic manipulator based on detected primitives."""
    grasp_plans = []
    for primitive_type, model in primitives:
        if primitive_type == "Plane":
            # Example: Grasp at the centroid of the plane
            centroid = model[:3]
            normal = model[3:]
            grasp_plans.append({
                "type": "Plane",
                "centroid": centroid,
                "approach_vector": normal
            })
        # Add logic for other shapes like spheres, cylinders, or cones

    return grasp_plans

def main():
    obj_file_path = "/home/ritz/Desktop/SEM 3/ML/Project/bottle.obj"  # Update file path

    # Step 1: Load and display the .obj file
    mesh = load_and_display_obj(obj_file_path)

    # Step 2: Reduce dimensions or filter relevant points
    reduced_points, _ = reduce_dimensions(mesh)
    filtered_points = filter_convex_hull(reduced_points)

    # Step 3: Detect primitives using Open3D
    primitives = detect_primitives_o3d(filtered_points)

    # Step 4: Fit Gaussian Mixture Model for segmentation
    labels, gmm = fit_mixture_of_gaussians(filtered_points, n_components=4)
    
    # Visualize clusters
    visualize_clusters(filtered_points, labels)

    # Step 5: Generate grasp plans based on detected primitives
    grasp_plans = generate_robotic_grasp_plan(primitives)
    for plan in grasp_plans:
        print("Grasp Plan:", plan)

if __name__ == "__main__":
    main()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x781706b746b0_12&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x78171c4ffd40_13&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x781706196390_14&reconnect=auto" class="pyv…

Grasp Plan: {'type': 'Plane', 'centroid': array([ 9.99999990e-01, -1.30789550e-04,  4.57816721e-05]), 'approach_vector': array([-0.05911607])}


In [4]:
import numpy as np
import pyvista as pv
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from scipy.spatial import ConvexHull
import open3d as o3d

def load_and_display_obj(file_path):
    """Load and display a .obj file."""
    mesh = pv.read(file_path)
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, color='white', show_edges=True)
    plotter.show()
    return mesh

def reduce_dimensions(mesh, n_components=3):
    """Reduce dimensions of the 3D points using PCA."""
    points = mesh.points  # Get the 3D points of the mesh
    pca = PCA(n_components=n_components)
    reduced_points = pca.fit_transform(points)
    
    # Visualize reduced data
    cloud = pv.PolyData(reduced_points)
    plotter = pv.Plotter()
    plotter.add_mesh(cloud, render_points_as_spheres=True, point_size=5)
    plotter.show()
    return reduced_points, pca

def filter_convex_hull(points):
    """Filter points by retaining only those on the convex hull."""
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]
    
    # Visualize convex hull
    cloud = pv.PolyData(hull_points)
    plotter = pv.Plotter()
    plotter.add_mesh(cloud, render_points_as_spheres=True, color="blue", point_size=5)
    plotter.show()
    return hull_points

def detect_primitives_o3d(points):
    """Detect shape primitives using Open3D."""
    primitives = []
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)

    # Plane detection using RANSAC
    plane_model, inliers = point_cloud.segment_plane(distance_threshold=0.01, ransac_n=3, num_iterations=1000)
    inlier_cloud = point_cloud.select_by_index(inliers)
    primitives.append(("Plane", plane_model, inlier_cloud))

    # Visualize plane detection
    o3d.visualization.draw_geometries([inlier_cloud], window_name="Detected Plane")

    # Add logic for sphere and other shapes if required
    return primitives

def fit_mixture_of_gaussians(points, n_components=4):
    """Fit a Gaussian Mixture Model to the points."""
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    gmm.fit(points)
    labels = gmm.predict(points)
    
    # Visualize GMM clusters
    cloud = pv.PolyData(points)
    cloud["Cluster"] = labels
    plotter = pv.Plotter()
    plotter.add_mesh(cloud, render_points_as_spheres=True, point_size=5)
    plotter.show()
    return labels, gmm

def generate_robotic_grasp_plan(primitives):
    """Generate a grasp plan for a robotic manipulator based on detected primitives."""
    grasp_plans = []
    for primitive_type, model, inlier_cloud in primitives:
        if primitive_type == "Plane":
            # Example: Grasp at the centroid of the plane
            centroid = np.mean(np.asarray(inlier_cloud.points), axis=0)
            normal = model[:3]
            grasp_plans.append({
                "type": "Plane",
                "centroid": centroid.tolist(),
                "approach_vector": normal.tolist()
            })

    # Visualize detected primitive grasp points
    for plan in grasp_plans:
        sphere = pv.Sphere(center=plan["centroid"], radius=0.02)
        plotter = pv.Plotter()
        plotter.add_mesh(sphere, color="red")
        plotter.show()

    return grasp_plans

def main():
    obj_file_path = "/home/ritz/Desktop/SEM 3/ML/Project/bottle.obj"  # Update file path

    # Step 1: Load and display the .obj file
    mesh = load_and_display_obj(obj_file_path)

    # Step 2: Reduce dimensions or filter relevant points
    reduced_points, _ = reduce_dimensions(mesh)
    filtered_points = filter_convex_hull(reduced_points)

    # Step 3: Detect primitives using Open3D
    primitives = detect_primitives_o3d(filtered_points)

    # Step 4: Fit Gaussian Mixture Model for segmentation
    labels, gmm = fit_mixture_of_gaussians(filtered_points, n_components=4)

    # Step 5: Generate grasp plans based on detected primitives
    grasp_plans = generate_robotic_grasp_plan(primitives)
    for plan in grasp_plans:
        print("Grasp Plan:", plan)

if __name__ == "__main__":
    main()


Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x781707fa6300_15&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x781705031eb0_16&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x781705045fd0_17&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x7816f4d1c560_18&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x7816f4de8890_19&reconnect=auto" class="pyv…

Grasp Plan: {'type': 'Plane', 'centroid': [0.059116067229013296, -2.0381963677468154e-05, 9.178742304338281e-06], 'approach_vector': [0.9999999903990661, -0.00013078954992101958, 4.578167208020358e-05]}


In [2]:
import numpy as np
import pyvista as pv
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from scipy.spatial import ConvexHull

def load_and_display_obj(file_path):
    # Read and visualize the 3D object
    mesh = pv.read(file_path)
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, color='white', show_edges=True)
    plotter.show()
    return mesh

def reduce_dimensions(mesh, n_components=3):
    # Reduce dimensions of the points in the mesh using PCA
    points = mesh.points
    pca = PCA(n_components=n_components)
    reduced_points = pca.fit_transform(points)

    # Visualize the reduced points
    reduced_cloud = pv.PolyData(reduced_points)
    plotter = pv.Plotter()
    plotter.add_mesh(reduced_cloud, render_points_as_spheres=True, point_size=5)
    plotter.show()

    return reduced_points, pca

def filter_convex_hull(points):
    # Compute and filter points on the convex hull
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]

    # Visualize convex hull points
    hull_cloud = pv.PolyData(hull_points)
    plotter = pv.Plotter()
    plotter.add_mesh(hull_cloud, color='blue', render_points_as_spheres=True, point_size=5)
    plotter.show()

    return hull_points

def fit_mixture_of_gaussians(points, n_components=4):
    # Apply Gaussian Mixture Model to fit and cluster points
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    gmm.fit(points)
    labels = gmm.predict(points)

    # Visualize GMM clustering results
    clustered_cloud = pv.PolyData(points)
    clustered_cloud["Cluster"] = labels
    plotter = pv.Plotter()
    plotter.add_mesh(clustered_cloud, render_points_as_spheres=True, point_size=5)
    plotter.show()

    return labels, gmm

def visualize_primitive_detection(points, labels):
    # Visualize detected primitives using the clustering labels
    cloud = pv.PolyData(points)
    cloud["Cluster"] = labels

    plotter = pv.Plotter()
    plotter.add_mesh(cloud, render_points_as_spheres=True, point_size=5)
    plotter.show()

def generate_grasp_plan(points, labels):
    # Visualize grasp plans for detected primitives
    cluster_centroids = []
    unique_labels = np.unique(labels)

    for label in unique_labels:
        cluster_points = points[labels == label]
        centroid = np.mean(cluster_points, axis=0)
        cluster_centroids.append(centroid)

        # Visualize each centroid
        sphere = pv.Sphere(center=centroid, radius=0.02)
        plotter = pv.Plotter()
        plotter.add_mesh(sphere, color="red")
        plotter.show()

    return cluster_centroids

def main():
    obj_file_path = "/home/ritz/Desktop/SEM 3/ML/Project/bottle.obj"

    # Step 1: Load and visualize the 3D object
    mesh = load_and_display_obj(obj_file_path)

    # Step 2: Reduce dimensions
    reduced_points, _ = reduce_dimensions(mesh)

    # Step 3: Filter convex hull points
    hull_points = filter_convex_hull(reduced_points)

    # Step 4: Fit Gaussian Mixture Model and visualize clusters
    labels, gmm = fit_mixture_of_gaussians(hull_points, n_components=4)

    # Step 5: Visualize primitive detection
    visualize_primitive_detection(hull_points, labels)

    # Step 6: Generate and visualize grasp plans
    centroids = generate_grasp_plan(hull_points, labels)
    for idx, centroid in enumerate(centroids):
        print(f"Grasp Plan {idx + 1}: Centroid at {centroid}")

if __name__ == "__main__":
    main()


Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x78173f6a21e0_3&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x781733f29df0_4&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x781733f3a150_5&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x7817381e81a0_6&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x7817381ea330_7&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x7817381ea4b0_8&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x781728d023f0_9&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x78171750c5f0_10&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:36239/index.html?ui=P_0x78171750e7e0_11&reconnect=auto" class="pyv…

Grasp Plan 1: Centroid at [ 1.08875677e-02 -1.40581724e-05  6.96515766e-06]
Grasp Plan 2: Centroid at [-0.13310048 -0.01764257  0.01608137]
Grasp Plan 3: Centroid at [-0.13549603  0.01225103 -0.01137257]
Grasp Plan 4: Centroid at [ 5.91160672e-02 -2.03819637e-05  9.17874230e-06]
